In [ ]:
import numpy as np
from netCDF4 import Dataset

In [ ]:
def rot2grid(rlon,rlat,rlon_min,rlat_min,dlon,dlat):
    x=(rlon-rlon_min)/dlon
    y=(rlat-rlat_min)/dlat
    return x,y

def interpol(x,y,var):
    x0=int(x)
    x1=x0+1
    y0=int(y)
    y1=y0+1

    w0=(x-x0)*(y-y0)
    w1=(x1-x)*(y-y0)
    w2=(x1-x)*(y1-y)
    w3=(x-x0)*(y1-y)
    vel = w0*var[y1,x1]+w1*var[y1,x0]+w2*var[y0,x0]+w3*var[y0,x1]
    return vel

In [ ]:
f=Dataset('data/z.nc','r')

rglon = f.variables['rlon'][:]
rglat = f.variables['rlat'][:]
gdlon = round(rglon[1]-rglon[0],4)
gdlat = round(rglat[1]-rglat[0],4)

f.close()

f=Dataset('data/l.nc','r')

surfpres = f.variables['PS'][:]
surftemp = f.variables['T_2M'][:]
groundtemp = f.variables['T_S'][:]
temp = f.variables['T'][:]
pres = f.variables['P'][:]
qv = f.variables['QV'][:]
qv_s = f.variables['QV_S'][:]
td_2m = f.variables['TD_2M'][:]
tke = f.variables['TKE'][:]

f.close()

f=Dataset('data/c.nc','r')

hhl = f.variables['HHL'][0,:,0,0]

f.close()




In [ ]:
# init zlev and add one extra level for the 2m values
zlevs=np.zeros(temp.shape[1]+1)
zlevs[temp.shape[1]]=2.

# fill zlev with values
for i in range(len(hhl)-1):
    zlevs[i]=hhl[i+1]+(hhl[i]-hhl[i+1])/2 - hhl[len(hhl)-1]


    
# def theta   
theta=np.zeros((temp.shape[0],temp.shape[1]+1,temp.shape[2],temp.shape[3]))

# calc theta 
for i in range(len(zlevs)-1):
    theta[:,i,:,:]=(temp[:,i,:,:]*(1+0.61*qv[:,i,:,:]))*(pres[:,len(zlevs)-2,:,:]/pres[:,i,:,:])**(287./1005.)


# calc theta 2m
e_2m=6.1078*np.exp(17.1*(td_2m-273.15)/(235+(td_2m-273.15)))
qv_2m=0.66*e_2m/surfpres
theta[:,len(zlevs)-1,:,:]=surftemp*(1+0.61*qv_2m)
    


In [ ]:
# load trajectory data
mean_traj = np.loadtxt('data/mean_traj.txt')
stda_traj = np.loadtxt('data/stda_traj.txt')